# **Import&path**

In [7]:
import zipfile as zf
files = zf.ZipFile("docs.zip", 'r')
files.extractall('./')
files.close()

In [8]:
import numpy as np
import pandas as pd
import copy
from gensim.models import Word2Vec
import keras
from numpy import dot
import math
from sklearn.preprocessing import normalize

file_path = './'

# **Read file**

In [31]:
## list
query_list = []
doc_list = []
all_query = []
all_doc = []
##query list
with open(file_path+'query_list.txt','r') as f:
    query = f.readlines()
    for i in range(len(query)):
        query_list.append(query[i].strip('\n'))

## doc list
with open(file_path+'doc_list.txt','r') as f:
    doc = f.readlines()
    for i in range(len(doc)):
        doc_list.append(doc[i].strip('\n'))
##all query
for i in range(len(query_list)):
    with open(file_path+'./queries/'+str(query_list[i])+'.txt','r') as f:
        q_content = f.readlines()
        all_query.append(q_content)
## all doc
for i in range(len(doc_list)):
    with open(file_path+'./docs/'+str(doc_list[i])+'.txt','r') as f:
        d_content = f.readlines()
        all_doc.append(d_content)
print(all_query[0])
print(len(all_doc[0]))

['intern organ crime']
1


In [32]:
doc_list.remove('LA072189-0048')
for i in range(len(doc_list)):
    if (doc_list[i]=='LA072189-0048'):
        print(i)
print(all_doc[3643])
del all_doc[3643:3644]
print(len(all_doc))

[]
4190


In [33]:
##試試新的
all_input = all_query + all_doc
column_name = []
all_data_split = [] ##分割字串
for i in range(len(all_input)):
    all_data_split.append(all_input[i][0].split())
## 讀入並掃次數
for i in range(len(all_data_split)):
    print(len(all_data_split[i]))
dot_dic = {} ##拿來乘的只有key的
for i in range(len(all_data_split)):
    for j in range(len(all_data_split[i])):
        dot_dic[all_data_split[i][j]] = 0.0
print('voc_dic done!')
query_dic = {} 
for i in range(50): ##query出現的字
    for j in range(len(all_data_split[i])):
        query_dic[all_data_split[i][j]] = 0.0
print(len(query_dic))
print('query_dic done!')
N = len(all_input) ##總共幾篇
score_dict_list = [] ##拿來存tf的所有權重
voc_dic = {} ##idf計數的
result = [] ##存結果的
for i in range(len(all_data_split)): ##幾篇文章 
    score_dic = {} ##算tf用的
    updated = {} ##已經算過的字
    for j in range(len(all_data_split[i])): ##所有的字
        if (voc_dic.get(all_data_split[i][j]) != None): ##已經存在
            if (all_data_split[i][j] not in updated):   ##這篇還沒看到
                voc_dic[all_data_split[i][j]] += 1.0
                updated[all_data_split[i][j]] = 1.0
        else:        ##沒有發現過這個字
            voc_dic[all_data_split[i][j]] = 1.0   #初始化
            updated[all_data_split[i][j]] = 1.0   #加入更新過的陣列
        if score_dic.get(all_data_split[i][j]) == None:
            score_dic[all_data_split[i][j]] = 1.0 ##建一個
        else:
            score_dic[all_data_split[i][j]] += 1.0 ##計數加一
    # example_dic[all_data_split[i][j]] = 0
    score_dict_list.append(score_dic) ##存了所有tf有值的分數
    if (i == len(all_data_split)-1): ##跑完了
        for k in range(len(score_dict_list)):
            for value in score_dict_list[k]:
                if (value in query_dic ):
                    dot_dic[value] = (1+math.log10(score_dict_list[k][value]))* math.log10(1+((N+1)/(voc_dic[value]+1)))
            score = list(dot_dic.values())
            result.append(score)
            for value in score_dict_list[k]:
                if (value in query_dic):
                    dot_dic[value] = 0.0
column_name = list(voc_dic.keys())
print('before norm')
result_score = normalize(result,norm ='l2')

3
3
3
3
2
3
3
2
2
4
2
1
3
2
3
3
2
3
3
4
2
3
3
3
2
2
2
2
3
3
3
3
3
3
3
3
2
2
3
3
2
2
2
3
3
2
2
1
1
3
224
527
141
373
5187
283
134
294
374
294
493
229
229
246
348
100
158
177
465
405
348
780
198
158
108
371
142
283
441
190
1656
284
323
612
145
176
735
613
140
441
294
414
329
1149
333
194
354
490
2114
229
400
86
251
1025
12154
4846
92
122
177
254
525
220
96
87
726
357
679
166
99
282
506
4053
108
219
282
398
165
443
227
299
505
500
651
290
129
1794
273
621
917
826
861
863
1491
2503
178
309
318
286
49
2209
1323
264
926
123
396
1912
276
178
419
274
5959
248
3293
704
1255
138
270
546
283
143
318
378
247
146
184
361
400
218
144
780
80
224
134
141
493
174
837
162
316
543
447
407
603
558
158
756
371
400
246
374
104
131
556
108
142
1660
681
190
290
462
749
694
480
138
294
188
145
440
2021
714
386
941
197
316
241
312
220
125
139
128
192
218
151
254
118
488
679
362
201
826
59
56
55
206
99
1125
276
241
229
517
245
17290
201
1304
502
43
53
356
465
37
285
221
1567
646
1651
162
670
601
150
676
832
760


voc_dic done!
123
query_dic done!
before norm


# **TF-IDF**

In [5]:
##會return 所有得分跟所有欄位
def my_tfidf(all_input):
    column_name = []
    all_data_split = [] ##分割字串
    for i in range(len(all_input)):
        all_data_split.append(all_input[i][0].split())
    print('split done!')
  ## 讀入並掃次數
    voc_dic = {}
    score_dict_list = [] ##拿來存tf的分數

    example_dic = {} ##拿來乘的空dict
    for i in range(len(all_data_split)): ##幾篇文章 
        score_dic = {} ##算tf用的
        updated = {} ##已經算過的字
        for j in range(len(all_data_split[i])): ##所有的字
            if (voc_dic.get(all_data_split[i][j]) != None): ##已經存在
                if (all_data_split[i][j] not in updated):   ##這篇還沒看到
                    voc_dic[all_data_split[i][j]] += 1.0
                    updated[all_data_split[i][j]] = 1.0
            else:        ##沒有發現過這個字
                voc_dic[all_data_split[i][j]] = 1.0   #初始化
                updated[all_data_split[i][j]] = 1.0   #加入更新過的陣列
            if score_dic.get(all_data_split[i][j]) == None:
                score_dic[all_data_split[i][j]] = 1.0 ##建一個
            else:
                score_dic[all_data_split[i][j]] += 1.0 ##計數加一
            example_dic[all_data_split[i][j]] = 0
        score_dict_list.append(score_dic) ##存了所有tf有值的分數
    column_name = list(voc_dic.keys())
  ## idf
    idf_dic = {} ##idf權重
    N = len(all_input) ##總共幾篇
    for i in voc_dic:
        idf_dic[i] = math.log(1+((N+1)/(voc_dic[i]+1))) ##sklearn文檔中的smooth_idf
    print('idf done!')
    result = []
    tf_dic = {} ##tf權重
    for i in range(N):
        print(i)
        for value in score_dict_list[i]:
            example_dic[value] =idf_dic[value]*(1+math.log(score_dict_list[i][value]))
        score = list(example_dic.values())
        result.append(score)
        for value in score_dict_list[i]:
            example_dic[value] = 0.0
    result_score = normalize(result,norm ='l2')
    return result_score,column_name

In [21]:
## 所有文檔 包括query跟doc
all_input = all_query+all_doc
# result_score,column_name = my_tfidf(all_input)
df_tfidf = pd.DataFrame(result_score,columns=column_name, index=(query_list+doc_list)) ##弄成dataframe
df_tfidf

,intern,organ,crime,poliomyel,post,polio,hubbl,telescop,achiev,endang,...,hammanskra,mavuso,mzondi,jabulani,14100,14600,holcomb,reigel,verna,caraccilo
301,0.389644,0.544460,0.742792,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
302,0.000000,0.000000,0.000000,0.753343,0.339882,0.562987,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.68936,0.671314,0.272251,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
304,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.502007,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
305,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LA123090-0026,0.000000,0.095488,0.000000,0.000000,0.000000,0.223643,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
LA123189-0136,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.034483,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
LA123190-0040,0.000000,0.000000,0.018654,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.057416,0.057416,0.057416,0.057416,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
LA123190-0062,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# **處理成斷詞的格式**

In [22]:
query_data = []
doc_data = []
for i in range(len(all_query)):
    query_data.append(all_query[i][0].split())

for i in range(len(all_doc)):
    if len(all_doc[i]) != 0:
        doc_data.append(all_doc[i][0].split())
print(query_data[0])
print(doc_data[0])

['intern', 'organ', 'crime']
['languag', 'f', 'p', '105', 'spanish', 'f', 'articl', 'type', 'bfn', 'text', 'santa', 'fe', 'de', 'bogota', '28', 'feb', 'dpa', 'today', 'colombian', 'prosecutor', 'gener', 'gustavo', 'de', 'greiff', 'said', 'u', 'govern', 'interest', 'suppli', 'evid', 'condemn', 'chief', 'drug', 'traffick', 'mafia', 'trust', 'colombian', 'justic', 'manner', 'de', 'greiff', 'reaffirm', 'difficulti', 'offic', 'encount', 'attempt', 'collect', 'evid', 'unit', 'state', 'main', 'capo', 'cali', 'cartel', 'sever', 'express', 'desir', 'surrend', 'collabor', 'author', 'colombia', 'desert', 'unit', 'state', 'interest', 'collabor', 'colombian', 'justic', 'although', 'judici', 'exchang', 'agreement', 'de', 'greiff', 'point', 'prosecutor', 'gener', 'ad', 'u', 'author', 'withhold', 'evid', 'reveal', 'later', 'mafioso', 'captur', 'countri', 'without', 'take', 'account', 'sever', 'think', 'surrend', 'colombia', 'opinion', 'director', 'colombian', 'justic', 'system', 'govern', 'washington'

In [23]:
##找出Index
def find_index(input):
    max = -1.0
    id = 0
    for i in range(len(input)):
        if (input[i] > max):
            max = input[i]
            id = i
    return id

In [25]:
ans_list = []
for i in range(len(query_data)): ##總共幾個query
    print(i)
    score_list = []       ##每個cos_sim的得分
    query_ans_list = []   ##排序輸出結果
    query_vec = np.zeros((len(query_data[i]))) ##每個query都有N個字當作n維向量
    doc_vec = np.zeros((len(doc_data),len(query_data[i]))) ##然後把doc的每個維度讀出來
    for j in range(len(query_data[i])): ## query有幾個詞
        query_vec[j] = df_tfidf.iloc[i][query_data[i][j]] ##query中每個字的分數存成vec
        for k in range(len(doc_data)):    ##總共幾篇doc
            doc_vec[k][j] = df_tfidf.iloc[k+50][query_data[i][j]] ##拿query中的字去看每篇doc的得分 弄成vec
  ##都存好開始算每個分數
    for j in range(len(doc_data)):
        cos_sim = 0
        for k in range(len(query_data[i])):
            cos_sim += dot(dot(query_vec[k],doc_vec[j][k]),query_vec[k]) ## query_vec的權重平方
        cos_sim /= len(query_data[i])
        score_list.append(cos_sim)
        for k in range(len(doc_data)):##用function找出從高到低的分數並記錄
            pos = find_index(score_list)
            query_ans_list.append(doc_list[pos])
            score_list[pos] = -2
    ans_list.append(query_ans_list)

0


KeyboardInterrupt: 

## **棄用**

# **弄出結果**

In [ ]:
error_name = 'LA072189-0048' ##錯誤的檔案 把它塞在最低分
with open(file_path+'hw1_result_mytfidf_1023.txt','w') as f:
    f.write('Query,RetrievedDocuments\n')
    for i in range(50): 
        f.write(query_list[i])
        f.write(',')
        for j in range(len(ans_list[0])):
            f.write(ans_list[i][j])
            f.write(' ')
        f.write(error_name)
        f.write('\n')
    f.close()
